In [ ]:
from typing import Any, Dict, Iterator, List, Optional
from langchain_core.callbacks import (
    CallbackManagerForLLMRun,
)
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
from langchain_core.messages.ai import UsageMetadata
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from pydantic import Field, PrivateAttr, BaseModel
from cfgenerator import CFGeneratorCM
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import numpy as np
import snowflake.connector
import json
# Fixed: removed duplicate 'import' keyword
from datetime import datetime
import asyncio
from langchain_core.runnables import RunnableLambda
# Note: AIMessage and ToolMessage already imported above, removing duplicates
# from langchain_core.messages import AIMessage, ToolMessage
# Note: Optional and Any already imported above, removing duplicates
from typing import Sequence

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import create_react_agent, ToolNode, tool_node
# from langgraph.prebuilt import create_react_agent_state
from langchain_core.tools import tool
from langchain.tools import Tool
from typing import TypedDict, List
# Note: BaseMessage already imported above, removing duplicate
# from langchain_core.messages import BaseMessage

import pandas as pd

print("Done!")

In [ ]:
LLM_MODEL_ID = "md0005_openai_gpt4o"  # "md0002_openai_gpt3516k"
# gdk = CFGenAIGDK("uc0027_abct_rag_cl", "rag_cl", "ChatBot to assist Business")
USE_STATIC = True  # Use the static workflow generator

In [ ]:
class GAIPCall:

    def __init__(
        self,
        llm_model_id: str = "md0005_openai_gpt4o",
        def_max_tokens: int = 200,
        def_temperature: float = 0.4,
        expt_name: str = "uc0027_abct_rag_cl",
        expt_id: str = "rag_cl",
        expt_desc: str = "ChatBot to assist Business"
    ):
        self.MODEL_ID = llm_model_id
        self.MAX_TOKENS = def_max_tokens
        self.TEMP = def_temperature
        self.EXP_NAME = expt_name
        self.EXP_ID = expt_id
        self.EXP_DESC = expt_desc

        # Create an instance of the gdk!
        self.gdk = CFGeneratorCM(self.EXP_NAME, self.EXP_ID, self.EXP_DESC)

    def invoke_llm_response(self, prompt: Optional[str], **kwargs) -> str:
        """
        Invokes the gdk response from a model, given a prompt and set of parameters passed through
        kwargs
        """

        placeholders = {k: v for k, v in kwargs.items() if (
            k != 'max_tokens' or k != 'temperature' or k != 'user_query')}

        unpacked_vals = {k: v for k, v in kwargs.items() if (
            k == 'max_tokens' or k == 'temperature' or k == 'user_query')}

        max_tokens = kwargs.get('max_tokens', self.MAX_TOKENS)
        temperature = kwargs.get('temperature', self.TEMP)
        # we account for there being no value for the user_query attribute during this call
        user_query = kwargs.get('user_query', "")
        prompt = prompt.format(**placeholders)

        if user_query != "":
            prompt_template = {
                "prompt_template": [
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": user_query}
                ]
            }
        else:
            prompt_template = {
                "prompt_template": [{"role": "system", "content": prompt}]
            }

        response = self.gdk.invoke_llmgateway(
            prompt_template,
            {"max_tokens": max_tokens, "temperature": temperature},
            self.MODEL_ID
        )

        content_filter_results = response["genResponse"]["choices"][0]["content_filter_results"]
        ai_message = response["genResponse"]["choices"][0]["message"]["content"]

        return ai_message


print("Done defining gaip services class")

In [ ]:
class AgentState(TypedDict, total=False):
    messages: List[BaseMessage]
    use_static: Optional[str]
    workflow: Optional[Any] = None
    tool_name: Optional[str]
    tool_input: Optional[str]
    tool_number: Optional[int] = 0
    done: Optional[bool] = False


print("Done defining the agent state class TypedDict")

In [ ]:
class AgentOrchestrator(BaseChatModel):
    _LLM_MODEL_ID: str = PrivateAttr()
    _DEF_MAX_TOKENS: str = PrivateAttr()
    _DEF_TEMPERATURE: str = PrivateAttr()
    _expt_name: str = PrivateAttr()
    _expt_id: str = PrivateAttr()
    _expt_desc: str = PrivateAttr()
    USE_STATIC: bool = PrivateAttr()
    _gaip = PrivateAttr()

    def __init__(
        self,
        llm_model_id: str = "md0005_openai_gpt4o",
        def_max_tokens: int = 200,
        def_temperature: float = 0.4,
        expt_name: str = "uc0027_abct_rag_cl",
        expt_id: str = "rag_cl",
        expt_desc: str = "ChatBot to assist Business",
        use_static: bool = True,
        **kwargs
    ):
        # Makes sure all keyword params are inherited from the parent BaseChatModel class
        super().__init__(**kwargs)

        self._LLM_MODEL_ID = llm_model_id
        self._DEF_MAX_TOKENS = def_max_tokens
        self._DEF_TEMPERATURE = def_temperature
        self._expt_name = expt_name
        self._expt_id = expt_id
        self._expt_desc = expt_desc

        # Toggle between static and dynamic workflows
        # use_static: True generates a workflow with tool queue at the beginning and follows the steps until the end
        # use_static: False Dynamically decides what tool to use next at every step depending on the response of the previous tool and other inputs
        self.USE_STATIC = use_static

        # Create an instance of the GAIPCall class
        self._gaip = GAIPCall(
            self._LLM_MODEL_ID,
            self._DEF_MAX_TOKENS,
            self._DEF_TEMPERATURE,
            self._expt_name,
            self._expt_id,
            self._expt_desc
        )

    def _generate(self, messages: List[BaseMessage], **kwargs) -> ChatResult:
        """
        DYNAMIC GENERATE: the model chooses the tool and the input to be given to the tool to be invoked
        Invokes the gdk's function to return a response and adds the response to AIMessage
        """

        if self.USE_STATIC:

            # Fetch only the latest human message for intent identification

            # CALL THE STATIC GENERATE FUNCTION
            response = self.static_generate(messages)

            return response

    def id_intent_workflow(self, messages: List[BaseMessage]) -> str:
        """
        Identify the intent of the user message to choose the most appropriate workflow to use
        """

        # prompt to identify the intent
        intent_followup_prompt = """
        Your task is to consider the user's message and identify two things based on it:
        1. The intent behind the message      
        2. If the question is a followup to a previous one
        
        Task 1: To identify the intent behind the messages, follow these guidelines:
        - PCT Query: If the user's message is about payment details, payment trends, transaction sums, 
          averages, mins and maxs, AML checks and fraud checks, the intent is PCT Query.
        - Weather: If the user's message is about the weather of any place, the intent is simply Weather
        - Miscellaneous: If the user's message is not about any of the above, the intent is Miscellaneous
        
        Task 2: To identify if the user message is a followup to a previous user message, use these guidelines:
        - Start by considering the previous few pairs of user messages and their AI responses
        
        Respond in this format and with no additional text or markdown if you identify that this message is a 
        followup to a previous user message
        {{
            intent: <identified intent>,
            followup_to: <prev user message this message is a followup to>,
            prev_ai_msg: <ai response to prev user message this message is a followup to>,
        }}
        
        Response in this format and with no additional text or markdown if you identify that this message is 
        not a followup to any previous user message
        {{
            intent: <identified intent>,
            followup_to: None,
            prev_ai_msg: None
        }}
        """

        intent_prompt = """
        Your task is to consider the user's message and identify the intent behind the message.
        
        To identify the intent behind the messages, follow these guidelines:
        - PCT Query: If the user's message is about payment details, payment trends, transaction sums, 
          averages, mins and maxs, AML checks and fraud checks, the intent is PCT Query.
        - Weather: If the user's message is about the weather of any place, the intent is simply Weather
        - Miscellaneous: If the user's message is not about any of the above, the intent is Miscellaneous
        
        Respond in this format and with no additional text or markdown
        {{
            "intent": <identified intent>
        }}
        """

        human_messages = [
            m.content for m in messages if isinstance(m, HumanMessage)]
        last_user_message = human_messages[-1]
        prev_user_message = human_messages[-2] if len(
            human_messages) > 1 else ""

        params = {"max_tokens": 50, "temperature": 0.1,
                  "user_query": last_user_message}
        # Identify the intent
        ai_response_intent = self._gaip.invoke_llm_response(
            intent_prompt,
            **params
        )

        try:
            ai_response_json = json.loads(ai_response_intent)
        except json.JSONDecodeError as e:
            print(
                "Something went wrong when trying to convert the ai response string into a JSON!")

        print("ai_response_intent after self.gaip.invoke_llm_response():")
        # If the intent is PCT Query, return the prompt to generate the workflow
        if ai_response_json["intent"].lower() == "pct query":
            workflow_prompt = """
            Your task is to create a workflow JSON provided to you as input and generate a workflow JSON.
            The workflow must be structured like in the provided format. Your response must only contain the valid JSON and nothing 
            else, no additional text or markdown or anything.
            
            EXAMPLE JSON FORMAT:           
            {{
                "output": {{
                    "intent": "PCT Query",
                    "workflow": [
                        {{
                            "step": 1,
                            "type": "llm",
                            "function_name": "nl_to_sql",
                            "prompt": "Convert the following natural language query into a SQL statement to be executed on a Snowflake DB: '<USER QUERY>'",
                            "payload": {{
                                "user_query": "<USER QUERY>"
                            }}
                        }},
                        {{
                            "step": 2,
                            "type": "database",
                            "function_name": "snowflake_executor",
                            "payload": {{
                                "sql": "<sql output>"
                            }}
                        }},
                        {{
                            "step": 3,
                            "type": "llm",
                            "function_name": "response_formatter",
                            "prompt": "Use the data from the Snowflake sql query response provided to synthesize a natural language response to the user's original query: '<USER QUERY>'",
                            "payload": {{
                                "raw_data": "snowflake_executor_output",
                                "user_query": "<USER QUERY>"
                            }}
                        }}
                    ]
                }}
            }}
            
            While generating your valid JSON workflow, follow these rules:
            - Take care to replace all placeholders within the workflow with the actual value expected in those spots.
            - As long as the user's query is related to payment details, transactions, fraud checks, AML checks, the intent must always be 'PCT Query'
            - The entire workflow must always contain 3 steps:
            - These are the three steps that must be contained within the list against the 'workflow' attribute:
            - Step 1:
                - The "step" must be "1"
                - The "type" must be "llm"
                - The "function_name" must be "nl_to_sql"
                - The "prompt" must be with "Convert the following natural language query into a SQL statement to be executed on a Snowflake DB: <USER QUERY>" with the placeholder replaced by the actual user query
                - Finally, the "payload" must be a dictionary with the attribute "user_query" containing against it the actual user's query instead of "<USER QUERY>"
            - Step 2:
                - The "step" must be "2"
                - The "function_name" must be "snowflake_executor"
                - The "type" must be "database"
                - Finally, the "payload" must be a dictionary with the attribute "sql" containing the string "step_1_output.sql"
            - Step 3:
                - The "step" must be "3"
                - The "type" must be "llm"
                - The "function_name" must be "response_formatter"
                - The "prompt" must be "Use the data from the Snowflake sql query response provided to synthesize a natural language response to the user's original query: '<USER QUERY>'"
                - Finally, the "payload" must have the "raw_data" containing the string "step_2_output.data" and "user_query" containing the actual user query instead of "<USER QUERY>"
            """

        elif ai_response_json["intent"].lower() == "weather":
            workflow_prompt = """
            Your task is to identify the name of a city in the user's query and respond with a JSON like in example provided below and nothing else.
            Make sure to substitute all the placeholders indicated by "< >" with the appropriate values instead.
            
            {{
                "output": {{
                    "intent": "Weather",
                    "workflow": [
                        {{
                            "step": 1,
                            "type": "llm",
                            "function_name": "get_weather",
                            "prompt": "Understand the user's question, identify the name of the city in the query and use it to return this descriptive string in response: 'The weather in <identified city> is partly sunny with a high of 80°F.'",
                            "payload": {{
                                "user_query": "<USER QUERY>"
                            }}
                        }}
                    ]
                }}
            }}
            """

        else:
            workflow_prompt = """
            Your task is to construct and return this JSON, substituting the placeholder <USER QUERY> with the actual user query.              
            {{
                "output": {{
                    "intent": "Miscellaneous",
                    "workflow": [
                        {{
                            "step": 1,
                            "type": "llm",
                            "function_name": "general_response",
                            "prompt": "Understand the user's question and reply with at most 3 sentences. If you do not understand the question or do not have sufficient information about it, respond with 'I am sorry, I cannot help you with that.'",
                            "payload": {{
                                "user_query": "<USER QUERY>"
                            }}
                        }}
                    ]
                }}
            }}
            """

        # Generate the workflow using the prompt
        params = {
            "max_tokens": 1200,
            "temperature": 0.3,
            "user_query": last_user_message
        }
        ai_response = self._gaip.invoke_llm_response(
            workflow_prompt,
            **params
        )

        return ai_response  # for which should contain a valid JSON

    def static_generate(self, messages: List[BaseMessage], **kwargs) -> ChatResult:
        """
        Use the static thing generator does not return anything, it just loads to be invoked for a given input at
        once in the form of a loop - structure provided in example"""

        workflow_string = self.id_intent_workflow(
            messages)  # this returns a string for now

        # Check if the response is a valid JSON before passing it on...
        try:
            workflow_json = json.loads(workflow_string)
            except_json_valid_workflow_generated = True
        except json.JSONDecodeError as e:
            print(
                "JSONDecodeError when trying to validate the AI generated JSON workflow: {e}")
            # CONTINUE: Does it make sense to attempt again? If yes, what is the exit strategy?
            # For now:
            except_json_valid_workflow_generated = False

        message = AIMessage(
            content="I apologize, but I encountered an error while processing your request.",
            response_metadata={
                "source_type": "gaip_model",
                "model_name": "static_generate"
            }
        )

        generation = ChatGeneration(message=message)
        generations = [generation]

        ai_msg = ChatResult(generations=generations)
        return ai_msg

    def bind_tools(self, tools: Sequence[Any]) -> "AgentOrchestrator":
        """Creates a new instance of the AgentOrchestrator class with the bound_tools attribute and returns it"""
        print("** bind_tools called! List of tools passed:",
              [t.name for t in tools])
        new_instance = self.__class__(
            llm_model_id=self._LLM_MODEL_ID,
            def_max_tokens=self._DEF_MAX_TOKENS,
            def_temperature=self._DEF_TEMPERATURE,
            expt_name=self._expt_name,
            expt_id=self._expt_id,
            expt_desc=self._expt_desc,
            use_static=self.USE_STATIC
        )
        object.__setattr__(new_instance, "_LLM_MODEL_ID", self._LLM_MODEL_ID)
        object.__setattr__(new_instance, "_DEF_MAX_TOKENS",
                           self._DEF_MAX_TOKENS)
        object.__setattr__(new_instance, "_DEF_TEMPERATURE",
                           self._DEF_TEMPERATURE)
        object.__setattr__(new_instance, "_expt_name", self._expt_name)
        object.__setattr__(new_instance, "_expt_id", self._expt_id)
        object.__setattr__(new_instance, "_expt_desc", self._expt_desc)
        object.__setattr__(new_instance, "USE_STATIC", self.USE_STATIC)
        object.__setattr__(new_instance, "_gaip", self._gaip)
        object.__setattr__(new_instance, "bound_tools", tools)
        return new_instance

    @property
    def _llm_type(self) -> str:
        """
        """
        self._param_name = "uc0027_abct_rag_cl"
        self._param_id = "rag_cl"
        self._param_desc = "ChatBot to assist Business"
        self._gdk = CFGeneratorCM(
            self._param_name, self._param_id, self._param_desc)
        self._hyperparams = {
            "max_tokens": max_tokens,
            "temperature": temperature
        }
        self._model_id = llm_model_id

        return f"Citizens CFGGenAIGDK:{self._LLM_MODEL_ID}"

    print("Done!")

In [ ]:
# Custom tool #1
@tool
def get_weather(city: str) -> str:
    """A tool that returns the weather of a city. It accepts the name of a city as input and returns a
    string describing the weather in that city as output.
    """
    print("\\n===============================================================================\\n")
    print("\\n🌤️ Executing Get Weather Tool")

    return f"The weather in Dallas is sunny with a high of 85°F."


class getWeatherToolInput(BaseModel):
    city: str


get_weather_tool = Tool.from_function(
    func=get_weather,
    name="get_weather",
    description="Returns the weather for a given city.",
    args_schema=getWeatherToolInput
)

print("Done!")

In [ ]:
try:
    file_path = 'Docs/payment_data_dictionary_1.xlsx'
    df1 = pd.read_excel(file_path, sheet_name="payment_details")
    print("\\n===============================================================================\\n")
    print(df1)
    df2 = pd.read_excel(file_path, sheet_name="payment_metrics")
    print("\\n===============================================================================\\n")
    print(df2)
    df3 = pd.read_excel(file_path, sheet_name="payment_kpi_daily_summary")
    print("\\n===============================================================================\\n")
    print(df3)
    schema_data = f"""Schema and column descriptions of PAYMENT_DETAILS table:\\n
    {df1}
    
    Schema and column descriptions of PAYMENT_METRICS table:
    {df2}
    
    Schema and column descriptions of PAYMENT_KPI_DAILY_SUMMARY table:
    {df3}
    """
except Exception as e:
    print(
        f"An Exception occurred when trying to read the payment_data_dictionary file: {e}")

schema_data

In [ ]:
@tool
def nl_to_sql(tool_input) -> str:
    """
    Converts a natural language question into a Snowflake-compatible SQL query. Accepts the user's question as input, and returns the generated SQL query as output.
    Previous exchange: (hist_context)
    """

    print("\\n===============================================================================\\n")
    # user_query = input_payload["user_query"]
    print("\\n🔍 Executing NL to SQL Tool")
    print(f"\\nAccepted TOOL INPUT: string <User's query>")
    print(f"TOOL INPUT RECEIVED: {tool_input}\\nTYPE: {type(tool_input)}")

    # No need to parse this input..
    user_query = tool_input

    # today
    today_date = datetime.now()
    date = today_date.strftime("%Y-%m-%d")
    day = today_date.strftime("%A")

    system_prompt = f"""
    You are a Snowflake SQL expert. Convert the following natural language question into a valid SQL query.
    Return only the generated SQL query in your response, no additional text or explanations. Make sure to
    follow the rules while generating the SQL query, and use the schema for context - this will help you
    generate the most relevant queries for the user's input.
    
    Rules:
    - ONLY USE SELECT QUERIES
    - DO NOT USE UPDATE, INSERT, DELETE, JOIN in your queries
    - Use proper Snowflake SQL syntax
    - Use table and column names exactly as shown in the schema
    - When specific dates or days are mentioned in the query, pay special attention to them and format the
      query with the right date or range of dates by calculating the correct values to go in the query based
      on today's date and day provided to you. For example, if the user asks for information about the previous
      Tuesday, consider the date and day it is today and count back to the previous Tuesday to deduce the correct
      date to be used in the query.
    - Add LIMIT 20 for queries that might return many rows or pick the latest 20 rows
    - Use uppercase for SQL keywords like SELECT, FROM, WHERE, etc.
    - Table names should be unquoted if they don't contain special characters
    - Don't use limit for aggregations LIMIT
    - Return ONLY the SQL query, no explanations or markdown
    
    Tables' Schema:
    {schema_data}
    
    Today's date for reference:
    {date}
    
    Today's day for reference:
    {day}
    """

    hyper_params = {
        "max_tokens": 300,
        "temperature": 0.1
    }
    gdk = CFGGenAIGDK("uc0027_abct_rag_cl", "rag_cl",
                      "ChatBot to assist Business")
    prompt_template = {
        "prompt_template": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_query
            }
        ]
    }
    response = gdk.invoke_llmgateway(
        prompt_template,
        hyper_params,
        LLM_MODEL_ID
    )

    generated_SQL = response["genResponse"]["choices"][0]["message"]["content"]

    return generated_SQL


class nl2SQLToolInput(BaseModel):
    user_query: str


nl_to_sql_tool = Tool.from_function(
    func=nl_to_sql,
    name="nl_to_sql",
    description="Returns a Snowflake compatible SQL query for the PCI Database.",
    args_schema=nl2SQLToolInput
)

print("Done!")

In [ ]:
# Custom Tool #3 - SQL execution tool
def create_snowflake_connection():
    try:
        conn = snowflake.connector.connect(
            user='SNOWFLAKE_PCI_PE_RO_P2',
            password='MhgfrwqcvfdR6mgte',
            account='citizensbank-p2.privatelink',
            warehouse='BCT_WH',
            database='BUSINESS_CONTROL_TOWER',
            schema='PAYMENT_CONTROL_TOWER',
            role='SNOWFLAKE_ROLE_PCI_PE_RO_P2'
        )
        print("Connected to Snowflake")
        return conn
    except Exception as e:
        print(f"Snowflake connection failed: {e}")
        return None


def execute_snowflake_query(query, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        data = cursor.fetchall()
        cursor.close()
        return data
    except Exception as e:
        print(f"Query Execution Error: {e}")
        return None


@tool
def execute_snowflake_sql(tool_input) -> str:
    """A tool that accepts a SQL query as input, executes it on a Snowflake DB through
    an API connection and returns the response from the query as output"""
    print("\\n===============================================================================\\n")
    print("\\n🗄️ Executing Snowflake SQL Tool")
    # sql_query = input_payload["sql_query"]
    print(f"\\nExpected TOOL INPUT: string <SQL query>")
    print(f"TOOL INPUT RECEIVED: {tool_input}\\nTYPE: {type(tool_input)}")

    conn = create_snowflake_connection()
    if not (tool_input.startswith('SELECT') or tool_input.startswith('WITH')):
        return "Unable to execute sql_query. No response returned"
    result = execute_snowflake_query(tool_input, conn)
    print(f"\\nResult of executed SQL query:")
    print(result)
    conn.close()
    return str(result)  # wrap the result into a string


class sqlExecutionToolInput(BaseModel):
    sql_query: str


sql_execution_tool = Tool.from_function(
    func=execute_snowflake_sql,
    name="execute_snowflake_sql",
    description="Executes a sql query on Snowflake and returns the results.",
    args_schema=sqlExecutionToolInput
)

print("Done defining the SQL execution tool")

In [ ]:
# Custom tool #4 - response synthesizer

@tool
def synthesize_response(tool_input: dict) -> str:
    """A tool that accepts context containing the user's natural language query, the SQL query generated for it,
    as well as the results of the executed SQL query from the Snowflake database, and synthesizes a final human
    comprehensible response as output to the user's question."""

    print("\\n===============================================================================\\n")
    print("\\n🧠 Executing Synthesize Response Tool")

    raw_data = tool_input["raw_data"]
    user_query = tool_input["user_query"]

    system_prompt = f"""
    You are a helpful AI assistant whose task is to consider the user's query, \\
    and the response of its corresponding SQL query execution on a Snowflake database \\
    to synthesize a final response which is human comprehensible and easy to read.
    
    When responding, keep in mind that you are answering the user's query; \\
    and the results of SQL query execution contain all the information you will require to \\
    answer the user's query in a meaningful way.
    
    Results of SQL query execution as context for your final response:
    {raw_data}
    
    If the SQL execution result has returned multiple rows of structured data in the form of tuples \\
    within a list, format your response to look like this by separating out individual values: \\
    
    | column 1      | column 2      | column 3      | column 4      |
    |---------------|---------------|---------------|---------------|
    | data          | data          | data          | data          |
    | data          | data          | data          | data          |
    
    If the result of the sql query execution is a single row or a count value, just respond with a single \\
    descriptive sentence.
    """

    hyper_params = {
        "max_tokens": 2000,
        "temperature": 0.35
    }
    prompt = {
        "prompt_template": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_query
            }
        ]
    }
    model_id = LLM_MODEL_ID = "md0002_openai_gpt3516k"  # "md0005_openai_gpt4omini"

    gdk = CFGeneratorCM("uc0027_abct_rag_cl", "rag_cl",
                        "ChatBot to assist Business")
    response = gdk.invoke_llmgateway(
        prompt,
        hyper_params,
        model_id
    )

    return response["genResponse"]["choices"][0]["message"]["content"]

    # class synthesizeResponseToolInput(BaseModel):
    # tool_input: Optional[Any]


synthesize_response_tool = Tool.from_function(
    func=synthesize_response,
    name="synthesize_response",
    description="Accepts the user's query, and the results of the corresponding SQL query execution, and returns a final synthesized response to the user's original query.",
)

print("Done!")

In [ ]:
# Create the one globally used instance of the class with the bind_tools in place

# CHANGE use_static = False FOR DYNAMIC TOOL CALLING
model = AgentOrchestrator(
    llm_model_id=LLM_MODEL_ID, def_max_tokens=200, def_temperature=0.4, expt_name="uc0027_abct_rag_cl",
    expt_id="rag_cl", expt_desc="ChatBot to assist Business", use_static=True
)

# llm_model_id=LLM_MODEL_ID, use_static=False, max_tokens=300)
model = model.bind_tools([
    nl_to_sql_tool,
    get_weather_tool,
    sql_execution_tool,
    synthesize_response_tool
])

print("Done initializing a model instance and binding all tools to it!")

In [ ]:
def end(state: AgentState) -> AgentState:
    print("\\n===============================================================================\\n")
    print("\\n Reached end of graph! Exiting.")
    return state


print("Done defining end")

In [ ]:
def check_done(state: AgentState) -> str:
    return "end" if state.get("done") else "interrupt"


print("Done defining the check done message")

In [ ]:
def interpret_v6(state: AgentState) -> AgentState:

    print("\\n===============================================================================\\n")
    print("\\n Interpreting...\\n")

    if state.get("use_static") is True and state.get("workflow", None) == None:

        # IF STATIC DO self.static_generate
        # response = self.static_generate(state["messages"])
        # ONLY DONE ONCE!!
        response = model.invoke(state["messages"])

        workflow_string = response.content  # JSON wrapped by str

        print("\\nWORKFLOW GENERATED:\\n")
        print(workflow_string)

        workflow_json = json.loads(workflow_string)

        # Update the state with the generated workflow - only once!
        state = {
            **state,  # preserve existing keys like "done"
            "workflow": workflow_json,
            "messages": state["messages"] + [response]
        }

    else:
        print("\\nNothing new to do inside interpret this time! Returning state unchanged")

    return state

In [ ]:
def use_tool_v7(state: AgentState) -> AgentState:
    """
    Function that calls one tool function - depending on the latest populated values of tool_to_use
    and tool_input from state
    This function should handle tool failures and exceptions!
    """

    print("\\n===============================================================================\\n")
    print("\\n🔧 Using Tool...\\n")

    # check if "done" has been set to True already and return state if so
    # print(f"\\nstate: {state}")
    if state.get("done", False):
        return state  # Return state, no changes done

    if state.get("tool_number", 0) == 0:
        state["tool_number"] = 0  # initialize to 0 is if not yet done

    # We have a valid state with tool_to_use and tool_input updated by plan_tool_use
    tool_name = state.get("tool_name")
    tool_input = state.get("tool_input")

    if tool_name == "nl_to_sql" and tool_input:
        # {"user_query": "What is the average number of transactions stopped for AML checks over the first week of September?"}
        print(f"\\n🔍 Calling tool {tool_name} with input: {tool_input}")
        sql_result = nl_to_sql.invoke({"user_query": tool_input})
        ai_message = AIMessage(
            content=sql_result,
            response_metadata={
                "source_type": "tool_call",
                "source_name": "nl_to_sql"
            }
        )
        print(f"\\n🟢 nl_to_sql Tool result: {sql_result}\\n")

        # Update the workflow JSON in state before returning it - the next tool should be able to
        # grab its input from that workflow..
        current_tool_number = state.get("tool_number")

        # if the current tool is not the last tool in the workflow, then update the payload of the
        # next tool with the output of this tool
        state["workflow"]["output"]["workflow"][current_tool_number +
                                                1]["payload"].update({"sql": sql_result})

        # Update tool Number count by one!
        state["tool_number"] += 1

        return {
            **state,
            "messages": state["messages"] + [ai_message]
        }

    elif tool_name == "get_weather" and tool_input:
        """
        structure of the tool_input received by this tool:
        tool_input = {"user_query": "user's natural language question>"}
        """
        # {"user_query": "What is the average number of transactions stopped for AML checks over the first week of September?"}
        user_query_abt_weather = tool_input["user_query"]

        print(
            f"\\n🌤️ Calling tool {tool_name} with input: {user_query_abt_weather}")
        weather_info = get_weather.invoke({"user_query_abt_weather"})
        ai_message = AIMessage(
            content=weather_info,
            response_metadata={
                "source_type": "tool_call",
                "source_name": "get_weather"
            }
        )
        print(f"\\n🟢 Get Weather TOOL output: {weather_info}\\n")

        # Update the workflow JSON in state before returning it - the next tool should be able to
        # grab its input from that workflow..
        current_tool_number = state.get("tool_number")

        # if the current tool is not the last tool in the workflow, then update the payload of the
        # next tool with the output of this tool
        if current_tool_number < len(state["workflow"]["output"]["workflow"]) - 1:
            state["workflow"]["output"]["workflow"][current_tool_number +
                                                    1]["payload"].update({"weather": weather_info})

        return {
            **state,
            "messages": state["messages"] + [ai_message],
            "done": True  # Optional: use for conditional edge to END
        }

    elif tool_name == "snowflake_executor" and tool_input:
        print(f"\n🔧 Calling tool '{tool_name}' with input: {tool_input}")
        query_exec_results = execute_snowflake_sql.invoke(tool_input["sql"])
        ai_message = AIMessage(
            content=query_exec_results,
            response_metadata={
                "source_type": "tool_call",
                "source_name": "execute_snowflake_sql"
            }
        )

        current_tool_number = state.get("tool_number")

        # If the current tool is not the last tool in the workflow, then update the payload of the
        # next tool with the output of this tool
        state["workflow"]["output"]["workflow"][current_tool_number +
                                                1]["payload"].update({"raw_data": query_exec_results})

        # Update Tool Number count by one!
        state["tool_number"] += 1

        return {
            **state,
            "messages": state["messages"] + [ai_message]
        }

    elif tool_name == "response_formatter" and tool_input:
        print(f"\n🔧 Calling tool '{tool_name}' with input: {tool_input}")
        final_response = synthesize_response.invoke({"tool_input": tool_input})
        ai_message = AIMessage(
            content=final_response,
            response_metadata={
                "source_type": "tool_call",
                "source_name": "synthesize_response"
            }
        )
        print(f"\n✅ Final synthesized response to user:\n{final_response}\n")

        return {
            **state,
            "messages": state["messages"] + [ai_message],
            "done": True  # Optional: use for conditional edge to END
        }

    else:
        print("⚠️ Tool not called - missing tool name or input.")
        error_msg = "Missing tool name or input. Returning."
        ai_message = AIMessage(
            content=error_msg,
            response_metadata={
                "source_type": "graph_node",
                "source_name": "use_tool"
            }
        )

        return {
            **state,
            "messages": state["messages"] + [ai_message],
            "done": True
        }


print("Done defining use_tool function")

In [ ]:
def plan_tool_use_v4(state: AgentState) -> AgentState:

    print("\n" + "="*70 + "\n")
    print("\n🧠 Planning Tool Use...")

    # If "done" has already been set to true for whatever reason, we continue onwards
    if state.get("done", False):
        return state

    if state["use_static"] is True:

        try:
            workflow_steps = state["workflow"]["output"]["workflow"]
            # list of only the pipeline names
            tool_queue = [step["pipeline_name"] for step in workflow_steps]
        except Exception as e:
            print(f"Unable to parse the workflow JSON. Error: {e}\n")
            return {
                **state,
                "tool_name": next_tool,
                "tool_input": tool_input
            }

        # Step 2: Fetch the name of the previous tool called from state, and the first tool if not yet set
        prev_tool = state.get("tool_name", None)
        print(f"\nPrev tool identified: {prev_tool}")

        # By this point we already have a tool queue
        if len(tool_queue) > 0:

            # Step 3: Find the name of the next tool to be called from the queue using the index of the prev tool
            # CONDITION FOR FIRST TOOL
            if prev_tool is None:
                next_tool = workflow_steps[0]["pipeline_name"]
                tool_input = workflow_steps[0]["payload"]
                state.update({"tool_number": 0})

            # CONDITION FOR LAST TOOL
            elif tool_queue.index(prev_tool) == len(tool_queue) - 1:
                next_tool = None
                print("\nDone servicing the entire queue!")
                return {
                    **state,
                    "tool_name": None,
                    "tool_input": None,
                    "done": True
                }

            # CONDITION FOR A TOOL IN BETWEEN
            else:
                tool_index = tool_queue.index(prev_tool)
                next_tool = workflow_steps[tool_index + 1]["pipeline_name"]
                tool_input = workflow_steps[tool_index + 1]["payload"]

            print(f"\nTool queue                    : {tool_queue}")
            tool_index = tool_queue.index(next_tool)
            print(f"\nIndex of next Tool in queue   : {tool_index}")
            print(f"\nNext Tool Name                : {next_tool}")
            print(f"\nNext Tool Input               : {tool_input}")

            return {
                **state,
                "tool_name": next_tool,
                "tool_input": tool_input
            }


print("plan_tool_use ready!")

In [ ]:
graph = StateGraph(AgentState)

# interpret_v5: _generate calls static_generate inside of it
graph.add_node("interpret", interpret_v6)
graph.add_node("plan_tool_use", plan_tool_use_v4)
graph.add_node("use_tool", use_tool_v7)  # contains the generate_workflow tool
graph.add_node("end", end)

graph.set_entry_point("interpret")
graph.add_edge("interpret", "plan_tool_use")
graph.add_edge("plan_tool_use", "use_tool")
# graph.add_edge("use_tool", "interpret")  # optional loop
# Forces langgraph to explicitly terminate the flow after reaching the end node
graph.add_edge("end", END)
graph.add_conditional_edges("use_tool", check_done, {
    "end": "end",
    "interpret": "interpret"
})

# add the memory as a checkpointed when compiling the graph
custom_agent = graph.compile()
print("Graph compiled and ready to go!")

In [ ]:
query1 = "Share the count of FED Transactions stopped for repair for the last Friday of the month of August 2025"
query2 = "Hey, what's the weather today?"
query3 = "How about the first Friday of September?"

response = custom_agent.invoke(
    {
        "messages": [
            SystemMessage(
                content="You are a helpful AI assistant. Use the provided tools to service the user's query."),
            HumanMessage(content=query1)
        ],
        "use_static": USE_STATIC,
        "done": False
    }
)
print("\nFinal response:")
ai_msgs = [m for m in response["messages"] if isinstance(m, AIMessage)]
last_ai_message = ai_msgs[-1].content
print(last_ai_message)